In [1]:
import numpy as np 
import pandas as pd 
import csv
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
sns.set_style("whitegrid")
from datetime import datetime, date
from os import path
import os
import sys

import keras
from keras.callbacks import EarlyStopping

In [2]:
# # load data, oversampled, 700 time units
# X_train_temporal = np.load('feature_temporal_cat_oversample_standardscalar_X_train.npy')[:,:700,:]
# X_train_cat = np.load('feature_temporal_cat_oversample_X_train.npy')[:,700:,1]

# X_test_temporal = np.load('features_temporal_test_norm_standardscalar.npy')
# X_test_cat = np.load('X_test_cat_features.npy')

# y_train = np.load('feature_temporal_cat_oversample_standardscalar_y_train.npy')
# y_test = np.load('y_test.npy')

In [3]:
# # load data, oversampled, 35 time units
X_train_temporal = np.load('feature_temporal_cat_agg_oversample_standardscalar_X_train.npy')[:,:35,:]
X_train_cat = np.load('feature_temporal_cat_agg_oversample_standardscalar_X_train.npy')[:,35:,1]

X_test_temporal = np.load('features_temporal_test_norm_agg_standardscalar.npy')
X_test_cat = np.load('X_test_cat_features.npy')

y_train = np.load('feature_temporal_cat_agg_oversample_standardscalar_y_train.npy')
y_test = np.load('y_test.npy')

In [4]:
print(X_train_temporal.shape)
print(X_test_temporal.shape)
print(X_train_cat.shape)
print(X_test_cat.shape)

(350, 35, 7)
(30, 35, 7)
(350, 68)
(30, 68)


In [5]:
from keras.models import Sequential, load_model

from keras.layers import Dense, Dropout
from keras.layers import LSTM
from keras.layers import Input
from keras.models import Model
from keras import regularizers
import keras

In [6]:
main_input = Input(shape=(X_train_temporal.shape[1], X_train_temporal.shape[2]), name='main_input')
lstm_out = LSTM(50, dropout=0.1, recurrent_dropout=0.1)(main_input)

auxiliary_input = Input(shape=(X_train_cat.shape[1],), name='aux_input')
# aux_1 = Dense(30, activation='relu')(auxiliary_input)
# aux_1= Dropout(0.1)(aux_1)
x = keras.layers.concatenate([lstm_out, auxiliary_input])

# stack a deep densely-connected network on top
# x = Dense(8, activation='relu')(x)
# x= Dropout(0.1)(x)
x = Dense(3, activation='relu')(x)
x= Dropout(0.1)(x)


# add the main logistic regression layer
main_output = Dense(1, activation='sigmoid', name='main_output')(x)
#This defines a model with two inputs:
model = Model(inputs=[main_input, auxiliary_input], outputs=[main_output])

In [7]:
# Model output shape
model.output_shape
# Model summary
model.summary()
# Model config
model.get_config()
# List all weight tensors 
model.get_weights()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
main_input (InputLayer)         [(None, 35, 7)]      0                                            
__________________________________________________________________________________________________
lstm (LSTM)                     (None, 50)           11600       main_input[0][0]                 
__________________________________________________________________________________________________
aux_input (InputLayer)          [(None, 68)]         0                                            
__________________________________________________________________________________________________
concatenate (Concatenate)       (None, 118)          0           lstm[0][0]                       
                                                                 aux_input[0][0]              

[array([[ 0.04612684,  0.12810898, -0.04923405, ...,  0.13690898,
         -0.01092963, -0.0668384 ],
        [-0.00968625,  0.11631796, -0.00224152, ..., -0.00431021,
          0.14576605,  0.14549384],
        [-0.1059039 , -0.12848085, -0.10927244, ..., -0.1245803 ,
         -0.13252679,  0.00027959],
        ...,
        [ 0.1469158 ,  0.04715782,  0.02248803, ...,  0.13195059,
          0.08830503,  0.16287541],
        [-0.00210904,  0.11532283,  0.12712407, ..., -0.05299948,
          0.06868903,  0.10984525],
        [-0.16858229,  0.1077919 , -0.16498628, ..., -0.09779214,
          0.1513162 ,  0.08627304]], dtype=float32),
 array([[-0.05075526, -0.02191966, -0.01147518, ..., -0.04134241,
         -0.01154169,  0.03756327],
        [ 0.0201015 ,  0.00538969,  0.08554165, ...,  0.03188874,
         -0.05273901,  0.02144886],
        [-0.08442931, -0.08168819, -0.00130725, ...,  0.0760088 ,
         -0.00403816, -0.07845777],
        ...,
        [-0.13860554, -0.01448477, -0.0

In [8]:
import keras.backend as K
from keras.callbacks import ReduceLROnPlateau,ModelCheckpoint
from keras.optimizers import adam_v2
def mean_pred(y_true, y_pred):
    return K.mean(y_pred)


opt = adam_v2.Adam(lr=0.0001)

model.compile(optimizer=opt,
              loss={'main_output': 'binary_crossentropy'},metrics=['accuracy',mean_pred])

reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.3, patience=5, min_lr=0.000001, verbose=1)
checkpointer = ModelCheckpoint(filepath='lstm.hdf5', verbose=1, save_best_only=True)

history=model.fit({'main_input': X_train_temporal, 'aux_input': X_train_cat},
          {'main_output': y_train},
          epochs=30, batch_size=128, callbacks=[reduce_lr, checkpointer],validation_split=0.1)

/Users/nanelia/opt/anaconda3/envs/my-project/lib/python3.8/site-packages/keras/optimizer_v2/optimizer_v2.py:357: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  warnings.warn(


Epoch 1/30
3/3 [==============================] - 18s 409ms/step - loss: 0.7614 - accuracy: 0.4833 - mean_pred: 0.5670 - val_loss: 0.5929 - val_accuracy: 0.5714 - val_mean_pred: 0.5582

Epoch 00001: val_loss improved from inf to 0.59288, saving model to lstm.hdf5
Epoch 2/30
3/3 [==============================] - 0s 30ms/step - loss: 0.7690 - accuracy: 0.4677 - mean_pred: 0.5728 - val_loss: 0.5955 - val_accuracy: 0.5714 - val_mean_pred: 0.5566

Epoch 00002: val_loss did not improve from 0.59288
Epoch 3/30
3/3 [==============================] - 0s 31ms/step - loss: 0.7499 - accuracy: 0.4588 - mean_pred: 0.5602 - val_loss: 0.5979 - val_accuracy: 0.5714 - val_mean_pred: 0.5552

Epoch 00003: val_loss did not improve from 0.59288
Epoch 4/30
3/3 [==============================] - 0s 30ms/step - loss: 0.7691 - accuracy: 0.4921 - mean_pred: 0.5674 - val_loss: 0.6003 - val_accuracy: 0.5143 - val_mean_pred: 0.5538

Epoch 00004: val_loss did not improve from 0.59288
Epoch 5/30
3/3 [===============

In [9]:
model.load_weights('lstm.hdf5')

In [10]:
y_score = model.predict({'main_input': X_test_temporal, 'aux_input': X_test_cat})

In [11]:
from sklearn.metrics import precision_score, average_precision_score,roc_auc_score

In [12]:
roc_auc_score(y_test, y_score)

0.7120000000000001

In [13]:
average_precision_score(y_test, y_score)

0.6055203619909502

In [14]:
from sklearn.metrics import classification_report
print(classification_report(y_test, (y_score>0.5).astype(int)))

              precision    recall  f1-score   support

           0       0.89      0.32      0.47        25
           1       0.19      0.80      0.31         5

    accuracy                           0.40        30
   macro avg       0.54      0.56      0.39        30
weighted avg       0.77      0.40      0.44        30

